# Laden der Daten

In [59]:
import pandas as pd
import numpy as np

COSTS = {
    "Moneycard": {"success": 5, "failed": 2},
    "Goldcard": {"success": 10, "failed": 5},
    "UK_Card": {"success": 3, "failed": 1},
    "Simplecard": {"success": 1, "failed": 0.5},
}
TIME_FEATURES = {"day": 31, "dow": 7, "hour": 24}

df = pd.read_excel("data.xlsx")
df = df.drop(columns=["Unnamed: 0"])


def process_data(processed: pd.DataFrame) -> pd.DataFrame:
    """Drop duplicates and generate Features."""
    processed = processed.drop_duplicates()
    processed = processed.copy()

    # Informationen aus Zeitstempel extrahieren
    processed["month"] = processed.loc[:, "tmsp"].dt.month.astype("int64")
    processed["week"] = processed.loc[:, "tmsp"].dt.isocalendar().week.astype("int64")
    processed["day"] = processed.loc[:, "tmsp"].dt.day.astype("int64")
    processed["dow"] = processed.loc[:, "tmsp"].dt.dayofweek.astype("int64")
    processed["hour"] = processed.loc[:, "tmsp"].dt.hour.astype("int64")
    processed["second"] = processed.loc[:, "tmsp"].dt.second.astype("int64")
    processed["is_weekend"] = np.where(processed["dow"] >= 5, True, False)
    processed["is_business_hours"] = np.where((processed["hour"] >= 8) & (processed["hour"] < 20), True, False)

    # Zeit-Features zyklisch kodieren
    # week und month nicht zyklisch kodieren da kein Zyklusübergang

    for key, value in TIME_FEATURES.items():
        processed[f"{key}_sin"] = np.sin(2 * np.pi * processed[key] / value)
        processed[f"{key}_cos"] = np.cos(2 * np.pi * processed[key] / value)

    # Kosten
    processed["cost"] = processed.apply(
        lambda row: COSTS[row["PSP"]]["success"] if row["success"] else COSTS[row["PSP"]]["failed"],
        axis=1,
    )

    # Wiederholte Transaktionsversuche aufgrund fehlgeschlagener Transaktionen
    processed["timedelta"] = processed["tmsp"].diff().dt.total_seconds().fillna(0).astype("int64")
    cols_to_compare = ["country", "amount", "3D_secured", "card"]
    processed["is_retry"] = (processed[cols_to_compare] == processed[cols_to_compare].shift(1)).all(axis=1)

    # anzahl kontinuierlicher Retry Versuche
    retry_groups = (~processed["is_retry"]).cumsum()
    processed["retry_count"] = processed.groupby(retry_groups)["is_retry"].cumsum().astype("int64")

    # Wechsel PSP bei Retry
    processed["PSP_switch"] = False
    processed["PSP_switch"] = np.where(
        (processed["is_retry"]) & (processed["PSP"] != processed["PSP"].shift(1))
        | processed["is_retry"] & (processed["PSP_switch"].shift(1)),
        True,
        False,
    )
    # anzahl aufeinanderfolgende failed unterschiedlicher Umsätze

    # anzahl
    return processed


processed = process_data(df)

In [57]:
pd.set_option("display.max_columns", None)
processed.iloc[100:150, :]

,tmsp,country,amount,success,PSP,3D_secured,card,month,week,day,dow,hour,second,is_weekend,is_business_hours,day_sin,day_cos,dow_sin,dow_cos,hour_sin,hour_cos,cost,timedelta,is_retry,retry_count,PSP_switch
101,2019-01-01 01:44:25,Germany,228,0,UK_Card,0,Diners,1,1,1,1,1,25,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,1.0,9,True,1,False
102,2019-01-01 01:45:17,Germany,228,1,UK_Card,0,Diners,1,1,1,1,1,17,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,3.0,52,True,2,False
103,2019-01-01 01:46:23,Germany,322,0,UK_Card,0,Master,1,1,1,1,1,23,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,1.0,66,False,0,False
104,2019-01-01 01:47:23,Germany,322,0,Moneycard,0,Master,1,1,1,1,1,23,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,2.0,60,True,1,True
105,2019-01-01 01:54:24,Austria,367,0,Moneycard,0,Visa,1,1,1,1,1,24,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,2.0,421,False,0,False
106,2019-01-01 01:55:08,Austria,367,0,UK_Card,0,Visa,1,1,1,1,1,8,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,1.0,44,True,1,True
107,2019-01-01 01:55:17,Austria,367,1,UK_Card,0,Visa,1,1,1,1,1,17,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,3.0,9,True,2,False
108,2019-01-01 01:57:28,Switzerland,190,0,UK_Card,0,Diners,1,1,1,1,1,28,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,1.0,131,False,0,False
109,2019-01-01 01:57:29,Switzerland,190,0,Simplecard,0,Diners,1,1,1,1,1,29,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,0.5,1,True,1,True
110,2019-01-01 01:58:00,Switzerland,190,0,UK_Card,0,Diners,1,1,1,1,1,0,False,False,0.201299,0.97953,0.781831,0.62349,0.258819,0.965926,1.0,31,True,2,True


In [5]:
processed[
    [
        "tmsp",
        "timedelta",
        "country",
        "amount",
        "3D_secured",
        "card",
        "PSP",
        "success",
        "is_retry",
    ]
].head(50)

,tmsp,timedelta,country,amount,3D_secured,card,PSP,success,is_retry
0,2019-01-01 00:01:11,999,Germany,89,0,Visa,UK_Card,0,False
1,2019-01-01 00:01:17,6,Germany,89,0,Visa,UK_Card,1,True
2,2019-01-01 00:02:49,92,Germany,238,1,Diners,UK_Card,0,False
3,2019-01-01 00:03:13,24,Germany,238,1,Diners,UK_Card,1,True
4,2019-01-01 00:04:33,80,Austria,124,0,Diners,Simplecard,0,False
5,2019-01-01 00:06:41,128,Switzerland,282,0,Master,UK_Card,0,False
6,2019-01-01 00:07:19,38,Switzerland,282,0,Master,Simplecard,0,True
7,2019-01-01 00:08:46,87,Germany,117,0,Master,UK_Card,1,False
8,2019-01-01 00:09:56,70,Switzerland,174,0,Visa,Simplecard,0,False
9,2019-01-01 00:10:49,53,Switzerland,174,0,Visa,Simplecard,0,True


In [ ]:
processed[
    [
        "tmsp",
        "timedelta",
        "country",
        "amount",
        "3D_secured",
        "card",
        "PSP",
        "success",
        "is_retry",
    ]
].to_csv("test.csv")

In [ ]:
pd.set_option("display.max_columns", None)
processed.loc[
    processed["is_retry"],
    [
        "tmsp",
        "timedelta",
        "country",
        "amount",
        "3D_secured",
        "card",
        "PSP",
        "success",
        "is_retry",
    ],
].head(50)

In [ ]:
time_df = processed.loc[processed["is_retry"], "timedelta"]
time_df.hist(bins=100)

In [ ]:
time_df = processed.loc[processed["is_retry"], ["timedelta", "is_retry"]]
time_df[processed["timedelta"] > 40].value_counts()

In [ ]:
time_df[processed["timedelta"] > 60]["timedelta"].unique()

In [ ]:
time_df[processed["timedelta"] > 60].shape